In [ ]:
import sys
sys.path.append("..")
%load_ext autoreload
%autoreload 2

In [ ]:
import folium

from src.main import MapItems, Locator, MapMaker

In [ ]:
locator = Locator()

In [ ]:
mm = MapMaker(map_item_filename='iceland_map_items.json')
mm.main()

In [ ]:
mm.base_map

In [ ]:
# Save the map to html file
mm.save_map('IcelandMap.html')

In [ ]:
# Save map icons (mi) to JSON file
mm.mi.to_json()

# Find locations

In [ ]:
locator("Videy", "iceland")

# Hotels

In [ ]:
locator("Borealis hotel", "iceland")

# Availability table
Create an overview table for the availability of hotels

In [ ]:
hotels = mm.mi.d['hotels']

In [ ]:
import re

In [ ]:
def parse_date1(s):
    regex = r"(\d+)\-(\d+) (Juli|Aug)"
    result = re.findall(regex, s)
    if len(result) == 0:
        print(f"1: Could not extract dates from {s}")
        return None
    else:
        return result[0]

def parse_date2(s):
    regex = r"(\d+) (Juli) \- (\d+) (Aug)"
    result = re.findall(regex, s)
    if len(result) == 0:
        print(f"2: Could not extract dates from {s}")
        return None
    else:
        return result[0]

In [ ]:
availability = dict()
for key, val in hotels.items():
    availability[key] = []
    for date in val['beschikbaarheid']:
        _date = parse_date1(date)
        if _date is None:
            _date = parse_date2(date)
        if _date is not None:
            availability[key].append(_date)

In [ ]:
juli = list(range(18, 32))
aug = list(range(1, 4))
_juli = ["Juli"] * (len(juli) - 1)
_aug = ["Aug"] * (len(aug) - 1)

In [ ]:
dates = list(zip(juli[:-1], _juli, juli[1:], _juli)) + [(juli[-1], _juli[0], aug[0], _aug[0])] + list(zip(aug[:-1], _aug, aug[1:], _aug))
fmt = "{} {} - {} {}"
dates = [fmt.format(*_date) for _date in dates]
# dates

In [ ]:
import pandas as pd
idx = pd.IndexSlice

In [ ]:
av = dict()
for key, val in availability.items():
    av[key] = []
    for date in val:
        if len(date) == 3:
            start_date, end_date, month = date
            start_month = month
            end_month = month
        else:
            start_date, start_month, end_date, end_month = date
        
        start_date = int(start_date)
        end_date = int(end_date)
        
        if start_date == end_date - 1:
            av[key].append(fmt.format(start_date, start_month, end_date, end_month))
        else:
            if start_month == end_month:
                ranges = list(range(start_date, end_date + 1))
                av[key].extend([fmt.format(*_date) for _date in list(zip(ranges[:-1], [start_month] * (len(ranges) - 1), ranges[1:], [end_month] * (len(ranges) - 1)))])
            else:
                ranges = list(range(start_date, 32))
                av[key].extend([fmt.format(*_date) for _date in list(zip(ranges[:-1], [start_month] * (len(ranges) - 1), ranges[1:], [start_month] * (len(ranges) - 1)))])
                
                av[key].append(fmt.format(31, start_month, 1, end_month))
                
                ranges = list(range(1, end_date + 1))
                av[key].extend([fmt.format(*_date) for _date in list(zip(ranges[:-1], [end_month] * (len(ranges) - 1), ranges[1:], [end_month] * (len(ranges) - 1)))])    

In [ ]:
df_av = dict()
for hotel, d in av.items():
    df_av[hotel] = [_date in d for _date in dates]
df = pd.DataFrame(index=dates, data=df_av).T
df = pd.concat([df, pd.DataFrame(data={key: [val['region']] for key, val in hotels.items()}).T], axis=1).rename(columns={0: "Regio"}).set_index("Regio", append=True).swaplevel(0, 1).sort_index()
df

In [ ]:
route = ["zw", "mid", "w", "n", "o", "zo", "z"]

In [ ]:
df.loc[idx[route, :]].astype(int)